In [ ]:
# time series doesn't have to be of a certain length

# to train an rnn, unroll through time and backpropagate
# backpropagation through time

# examples
# sequence to sequence rnn takes one sequence and outputs another ie stocks shifted 1 day in future
# sequence to vector : words fed in, score fed out
# vector to sequence : image to caption
# sequence to vector encoder followed by vector to sequence decoder: language translation

# keras will take care of unrolling and backpropagating through time

In [25]:
# time series forecasting

def generate_time_series(batch_size, n_steps):
    freq1, freq2, offsets1, offsets2 = np.random.rand(4, batch_size, 1)
    time = np.linspace(0, 1, n_steps)
    series = 0.5 * np.sin((time - offsets1) * (freq1 * 10 + 10))  #   wave 1
    series += 0.2 * np.sin((time - offsets2) * (freq2 * 20 + 20)) # + wave 2
    series += 0.1 * (np.random.rand(batch_size, n_steps) - 0.5)   # + noise
    return series[..., np.newaxis].astype(np.float32)

In [26]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

np.random.seed(42)

n_steps = 50
series = generate_time_series(10000, n_steps + 1)
X_train, y_train = series[:7000, :n_steps], series[:7000, -1]
X_valid, y_valid = series[7000:9000, :n_steps], series[7000:9000, -1]
X_test, y_test = series[9000:, :n_steps], series[9000:, -1]

# first, lets make sure we have a baseline
# lets just predict the last value in our training time series

y_pred = X_valid[:, -1]
np.mean(keras.losses.mean_squared_error(y_valid, y_pred))

0.020211367

In [27]:
# lets use simple model

model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[50, 1]),
    keras.layers.Dense(1)
])
model.compile(loss='mse', optimizer='adam')
history = model.fit(X_train, y_train, epochs=20,
                    validation_data=(X_valid, y_valid))


Epoch 1/20
219/219 [==============================] - 0s 1ms/step - loss: 0.1365 - val_loss: 0.0697
Epoch 2/20
219/219 [==============================] - 0s 958us/step - loss: 0.0475 - val_loss: 0.0326
Epoch 3/20
219/219 [==============================] - 0s 2ms/step - loss: 0.0246 - val_loss: 0.0187
Epoch 4/20
219/219 [==============================] - 0s 794us/step - loss: 0.0156 - val_loss: 0.0134
Epoch 5/20
219/219 [==============================] - 0s 779us/step - loss: 0.0120 - val_loss: 0.0111
Epoch 6/20
219/219 [==============================] - 0s 748us/step - loss: 0.0102 - val_loss: 0.0099
Epoch 7/20
219/219 [==============================] - 0s 746us/step - loss: 0.0092 - val_loss: 0.0090
Epoch 8/20
219/219 [==============================] - 0s 771us/step - loss: 0.0084 - val_loss: 0.0082
Epoch 9/20
219/219 [==============================] - 0s 762us/step - loss: 0.0077 - val_loss: 0.0076
Epoch 10/20
219/219 [==============================] - 0s 756us/step - loss: 0.0072 - 

In [28]:
# lets do a simple rnn

np.random.seed(42)
tf.random.set_seed(42)

model = keras.models.Sequential([
    keras.layers.SimpleRNN(1, input_shape=[None, 1])
])

optimizer = keras.optimizers.Adam(learning_rate=0.005)
model.compile(loss="mse", optimizer=optimizer)
history = model.fit(X_train, y_train, epochs=20,
                    validation_data=(X_valid, y_valid))

Epoch 1/20
219/219 [==============================] - 1s 5ms/step - loss: 0.0967 - val_loss: 0.0489
Epoch 2/20
219/219 [==============================] - 1s 4ms/step - loss: 0.0369 - val_loss: 0.0296
Epoch 3/20
219/219 [==============================] - 1s 4ms/step - loss: 0.0253 - val_loss: 0.0218
Epoch 4/20
219/219 [==============================] - 1s 4ms/step - loss: 0.0198 - val_loss: 0.0177
Epoch 5/20
219/219 [==============================] - 1s 4ms/step - loss: 0.0166 - val_loss: 0.0151
Epoch 6/20
219/219 [==============================] - 1s 4ms/step - loss: 0.0146 - val_loss: 0.0134
Epoch 7/20
219/219 [==============================] - 1s 4ms/step - loss: 0.0132 - val_loss: 0.0123
Epoch 8/20
219/219 [==============================] - 1s 5ms/step - loss: 0.0124 - val_loss: 0.0116
Epoch 9/20
219/219 [==============================] - 1s 4ms/step - loss: 0.0118 - val_loss: 0.0112
Epoch 10/20
219/219 [==============================] - 1s 4ms/step - loss: 0.0116 - val_loss: 0.0110

In [34]:
# lets make a deep rnn

np.random.seed(42)
tf.random.set_seed(42)

model = keras.models.Sequential([
    keras.layers.SimpleRNN(20, return_sequences=True, input_shape=[None, 1]),
    keras.layers.SimpleRNN(20, return_sequences=True),
    keras.layers.SimpleRNN(1)
])

model.compile(loss="mse", optimizer="adam")
history = model.fit(X_train, y_train, epochs=50,
                    validation_data=(X_valid, y_valid))

Epoch 1/50
219/219 [==============================] - 4s 13ms/step - loss: 0.0492 - val_loss: 0.0090
Epoch 2/50
219/219 [==============================] - 3s 12ms/step - loss: 0.0070 - val_loss: 0.0065
Epoch 3/50
219/219 [==============================] - 3s 11ms/step - loss: 0.0053 - val_loss: 0.0045
Epoch 4/50
219/219 [==============================] - 3s 12ms/step - loss: 0.0045 - val_loss: 0.0040
Epoch 5/50
219/219 [==============================] - 3s 13ms/step - loss: 0.0042 - val_loss: 0.0040
Epoch 6/50
219/219 [==============================] - 3s 12ms/step - loss: 0.0038 - val_loss: 0.0036
Epoch 7/50
219/219 [==============================] - 3s 12ms/step - loss: 0.0038 - val_loss: 0.0040
Epoch 8/50
219/219 [==============================] - 3s 12ms/step - loss: 0.0037 - val_loss: 0.0033
Epoch 9/50
219/219 [==============================] - 3s 12ms/step - loss: 0.0036 - val_loss: 0.0032
Epoch 10/50
219/219 [==============================] - 3s 12ms/step - loss: 0.0035 - val_lo

In [33]:
model = keras.models.Sequential([
    keras.layers.SimpleRNN(20, return_sequences=True, input_shape=[None, 1]),
    keras.layers.Dropout(.3),
    keras.layers.SimpleRNN(20, return_sequences=True),
    keras.layers.Dropout(.3),
    keras.layers.SimpleRNN(1)
])
opt = keras.optimizers.Adam(learning_rate=.001)
model.compile(loss="mse", optimizer=opt)
history = model.fit(X_train, y_train, epochs=20,
                    validation_data=(X_valid, y_valid))

Epoch 1/20
219/219 [==============================] - 4s 14ms/step - loss: 0.0873 - val_loss: 0.0084
Epoch 2/20
219/219 [==============================] - 3s 12ms/step - loss: 0.0223 - val_loss: 0.0053
Epoch 3/20
219/219 [==============================] - 3s 13ms/step - loss: 0.0137 - val_loss: 0.0046
Epoch 4/20
219/219 [==============================] - 3s 12ms/step - loss: 0.0112 - val_loss: 0.0041
Epoch 5/20
219/219 [==============================] - 3s 13ms/step - loss: 0.0099 - val_loss: 0.0039
Epoch 6/20
219/219 [==============================] - 3s 14ms/step - loss: 0.0085 - val_loss: 0.0039
Epoch 7/20
219/219 [==============================] - 3s 14ms/step - loss: 0.0083 - val_loss: 0.0042
Epoch 8/20
219/219 [==============================] - 3s 13ms/step - loss: 0.0077 - val_loss: 0.0038
Epoch 9/20
219/219 [==============================] - 3s 13ms/step - loss: 0.0072 - val_loss: 0.0036
Epoch 10/20
219/219 [==============================] - 3s 12ms/step - loss: 0.0073 - val_lo

In [35]:
np.random.seed(42)
tf.random.set_seed(42)

model = keras.models.Sequential([
    keras.layers.SimpleRNN(20, return_sequences=True, input_shape=[None, 1]),
    keras.layers.SimpleRNN(20),
    keras.layers.Dense(1)
])

model.compile(loss="mse", optimizer="adam")
history = model.fit(X_train, y_train, epochs=20,
                    validation_data=(X_valid, y_valid))

Epoch 1/20
219/219 [==============================] - 3s 10ms/step - loss: 0.0232 - val_loss: 0.0052
Epoch 2/20
219/219 [==============================] - 2s 9ms/step - loss: 0.0043 - val_loss: 0.0036
Epoch 3/20
219/219 [==============================] - 2s 9ms/step - loss: 0.0035 - val_loss: 0.0031
Epoch 4/20
219/219 [==============================] - 2s 9ms/step - loss: 0.0033 - val_loss: 0.0033
Epoch 5/20
219/219 [==============================] - 2s 9ms/step - loss: 0.0033 - val_loss: 0.0034
Epoch 6/20
219/219 [==============================] - 2s 9ms/step - loss: 0.0031 - val_loss: 0.0029
Epoch 7/20
219/219 [==============================] - 2s 9ms/step - loss: 0.0031 - val_loss: 0.0034
Epoch 8/20
219/219 [==============================] - 2s 9ms/step - loss: 0.0032 - val_loss: 0.0028
Epoch 9/20
219/219 [==============================] - 2s 9ms/step - loss: 0.0031 - val_loss: 0.0028
Epoch 10/20
219/219 [==============================] - 2s 9ms/step - loss: 0.0030 - val_loss: 0.002

In [39]:
# predict several time steps ahead

np.random.seed(42)
tf.random.set_seed(42)

# could just use the already made model to predict the next value then add it to the inputs and so on
series = generate_time_series(1, n_steps + 10)
X_new, y_new = series[:, :n_steps], series[:, n_steps:]
X = X_new
for step_ahead in range(10):
    y_pred_one = model.predict(X[:, step_ahead])[:, np.newaxis, :]
    print(y_pred_one)
    X = np.concatenate([X, y_pred_one], axis=1)
Y_pred = X[:, n_steps:]

[[[0.47165224]]]
[[[0.29770407]]]
[[[0.03169884]]]
[[[-0.14619076]]]
[[[-0.38052443]]]
[[[-0.4740657]]]
[[[-0.55154735]]]
[[[-0.402325]]]
[[[-0.3482777]]]
[[[-0.38091555]]]


In [45]:
np.random.seed(42)
tf.random.set_seed(42)
# could train rnn to output all 10 values at once
series = generate_time_series(10000, n_steps + 10)
print(series.shape)
print(series[0, :, 0])
X_train, Y_train = series[:7000, :n_steps], series[:7000, -10:, 0]
X_valid, Y_valid = series[7000:9000, :n_steps], series[7000:9000, -10:, 0]
X_test, Y_test = series[9000:, :n_steps], series[9000:, -10:, 0]

(10000, 60, 1)
[-0.64050776 -0.529444   -0.38806868 -0.19187537 -0.02863146  0.26193336
  0.3969534   0.6239992   0.6439498   0.6493094   0.6074959   0.43222332
  0.29762426  0.00236487 -0.20139012 -0.4558224  -0.6097667  -0.7264664
 -0.7349208  -0.57346964 -0.4505463  -0.2754545  -0.00935674  0.22663821
  0.39442402  0.55928534  0.6412419   0.6511253   0.5804723   0.43022403
  0.22126319  0.03515577 -0.22477219 -0.36486754 -0.573342   -0.6372609
 -0.65981853 -0.5331245  -0.48621044 -0.2896961  -0.09598922  0.13103025
  0.3539316   0.50686395  0.52818555  0.6129349   0.49392587  0.38097748
  0.28169915  0.07702279 -0.09526882 -0.3135954  -0.4068229  -0.5218387
 -0.51792175 -0.44039068 -0.40399557 -0.29317066 -0.08600036  0.03291251]


In [48]:
np.random.seed(42)
tf.random.set_seed(42)
# just output 10 units instead of 1
model = keras.models.Sequential([
    keras.layers.SimpleRNN(20, return_sequences=True, input_shape=[None, 1]),
    keras.layers.SimpleRNN(20),
    keras.layers.Dense(10)
])
model.compile(loss='mse', optimizer='adam')
model.fit(X_train, Y_train, epochs=20,
         validation_data=(X_valid, Y_valid))

Epoch 1/20
219/219 [==============================] - 3s 10ms/step - loss: 0.0656 - val_loss: 0.0320
Epoch 2/20
219/219 [==============================] - 2s 9ms/step - loss: 0.0272 - val_loss: 0.0222
Epoch 3/20
219/219 [==============================] - 2s 9ms/step - loss: 0.0194 - val_loss: 0.0161
Epoch 4/20
219/219 [==============================] - 2s 10ms/step - loss: 0.0159 - val_loss: 0.0147
Epoch 5/20
219/219 [==============================] - 2s 9ms/step - loss: 0.0138 - val_loss: 0.0125
Epoch 6/20
219/219 [==============================] - 2s 9ms/step - loss: 0.0126 - val_loss: 0.0116
Epoch 7/20
219/219 [==============================] - 2s 9ms/step - loss: 0.0119 - val_loss: 0.0116
Epoch 8/20
219/219 [==============================] - 2s 9ms/step - loss: 0.0117 - val_loss: 0.0103
Epoch 9/20
219/219 [==============================] - 2s 9ms/step - loss: 0.0109 - val_loss: 0.0101
Epoch 10/20
219/219 [==============================] - 2s 9ms/step - loss: 0.0109 - val_loss: 0.01

In [49]:
Y_pred = model.predict(X_new)
Y_pred

array([[ 0.4749645 ,  0.33652034,  0.19712791,  0.15423308,  0.13870144,
         0.14605808,  0.16045572,  0.13469052, -0.0137713 , -0.12791085]],
      dtype=float32)

In [55]:
# right now, is sequence to vector rnn
# we want sequence to sequence rnn to more accurately predict next 10 time steps

# only forecasts next 10 at very last step
# how about we forecast next 10 at each and every step
np.random.seed(42)
tf.random.set_seed(42)
Y = np.empty((10000, n_steps, 10))
for step_ahead in range(1, 11):
    Y[:, :, step_ahead - 1] = series[:, step_ahead:step_ahead + n_steps, 0]
Y_train = Y[:7000]
Y_valid = Y[7000:9000]
Y_test = Y[9000:]
print(Y.shape) # each is now 10 dimensions instead of 1

(10000, 50, 10)


In [56]:
# can output the 10 time steps using
np.random.seed(42)
tf.random.set_seed(42)
model = keras.models.Sequential([
    keras.layers.SimpleRNN(20, return_sequences=True, input_shape=[None, 1]),
    keras.layers.SimpleRNN(20, return_sequences=True),
    keras.layers.TimeDistributed(keras.layers.Dense(10))
])
# create metric to only compute mse on last time step

def last_time_step_mse(Y_true, Y_pred):
    return keras.metrics.mean_squared_error(Y_true[:, -1], Y_pred[:, -1])

optimizer = keras.optimizers.Adam(lr=.01)
model.compile(loss='mse', optimizer=optimizer, metrics=[last_time_step_mse])


In [ ]:
np.random.seed(42)
tf.random.set_seed(42)

model.fit(X_train, Y_train, epochs=20,
         validation_data=(X_valid, Y_valid))

Epoch 1/20
219/219 [==============================] - 3s 10ms/step - loss: 0.0502 - last_time_step_mse: 0.0385 - val_loss: 0.0420 - val_last_time_step_mse: 0.0282
Epoch 2/20
219/219 [==============================] - 2s 9ms/step - loss: 0.0398 - last_time_step_mse: 0.0284 - val_loss: 0.0375 - val_last_time_step_mse: 0.0275
Epoch 3/20
219/219 [==============================] - 2s 9ms/step - loss: 0.0338 - last_time_step_mse: 0.0228 - val_loss: 0.0315 - val_last_time_step_mse: 0.0207
Epoch 4/20
219/219 [==============================] - 2s 9ms/step - loss: 0.0299 - last_time_step_mse: 0.0184 - val_loss: 0.0286 - val_last_time_step_mse: 0.0164
Epoch 5/20
219/219 [==============================] - 2s 9ms/step - loss: 0.0279 - last_time_step_mse: 0.0159 - val_loss: 0.0273 - val_last_time_step_mse: 0.0150
Epoch 6/20
219/219 [==============================] - 2s 9ms/step - loss: 0.0257 - last_time_step_mse: 0.0134 - val_loss: 0.0260 - val_last_time_step_mse: 0.0120
Epoch 7/20
219/219 [=======